<a href="https://colab.research.google.com/github/danilo07florez/APISpringBoot/blob/main/RecuperarVacantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time
from google.colab import files


In [ ]:
departamentos = {
    "CORDOBA": "C%C3%93RDOBA",
    "LA_GUAJIRA": "LA+GUAJIRA",
    "MAGDALENA": "MAGDALENA",
    "CESAR": "CESAR",
    "ATLANTICO": "ATL%C3%81NTICO",
    "BOLIVAR": "BOL%C3%8DVAR",
    "SUCRE": "SUCRE",
    "NORTE_DE_SANTANDER": "NORTE+DE+SANTANDER",
    "SANTANDER": "SANTANDER",
    "ANTIOQUIA": "ANTIOQUIA",
    "CHOCO": "CHOC%C3%93",
    "RISARALDA": "RISARALDA",
    "CALDAS": "CALDAS",
    "QUINDIO": "QUINDIO",
    "CAUCA": "CAUCA",
    "NARINO": "NARI%C3%91O",
    "PUTUMAYO": "PUTUMAYO",
    "AMAZONAS": "AMAZONAS",
    "GUAVIARE": "GUAVIARE",
    "META": "META",
    "HUILA": "HUILA",
    "TOLIMA": "TOLIMA",
    "CUNDINAMARCA": "CUNDINAMARCA",
    "BOYACA": "BOYAC%C3%81",
    "ARAUCA": "ARAUCA",
    "CASANARE": "CASANARE",
    "VICHADA": "VICHADA",
    "GUAINIA": "GUAIN%C3%8DA",
    "VAUPES": "VAUP%C3%89S",
    "ISLAS": "ARCHIPI%C3%89LAGO+DE+SAN+ANDR%C3%89S%2C+PROVIDENCIA+Y+SANTA+CATALINA",
    "CAQUETA":"CAQUET%C3%81",
    "VALLE_DEL_CAUCA":"VALLE+DEL+CAUCA",
    "BOGOTA":"BOGOT%C3%81%2C+D.C.",
    "CAQUETA":"CAQUET%C3%81",
    "VALLE_DEL_CAUCA":"VALLE+DEL+CAUCA",
    "BOGOTA":"BOGOTÁ%2C+D.C."
}

BASE_URL = "https://www.buscadordeempleo.gov.co/backbue/v1/vacantes/resultados"


In [ ]:
def obtener_vacantes_departamento(dep_nombre, dep_query):
    todas = []
    page = 1

    while True:
        url = f"{BASE_URL}?page={page}&DEPARTAMENTO={dep_query}"
        response = requests.get(url, timeout=30)

        if response.status_code != 200:
            break

        data = response.json()
        resultados = data.get("resultados", [])

        if not resultados:
            break

        for r in resultados:
            r["DEPARTAMENTO_ORIGEN"] = dep_nombre

        todas.extend(resultados)
        page += 1
        time.sleep(0.2)  # evita sobrecargar el servidor

    print(f"{dep_nombre}: {len(todas)} vacantes")
    return todas


In [ ]:
vacantes_consolidadas = []

for nombre, query in departamentos.items():
    vacantes = obtener_vacantes_departamento(nombre, query)
    vacantes_consolidadas.extend(vacantes)

print("\nTotal vacantes consolidadas:", len(vacantes_consolidadas))


CORDOBA: 929 vacantes
LA_GUAJIRA: 585 vacantes
MAGDALENA: 1307 vacantes
CESAR: 936 vacantes
ATLANTICO: 8933 vacantes
BOLIVAR: 3668 vacantes
SUCRE: 449 vacantes
NORTE_DE_SANTANDER: 1680 vacantes


ConnectionError: HTTPSConnectionPool(host='www.buscadordeempleo.gov.co', port=443): Read timed out.

In [ ]:
archivo_salida = "vacantes_colombia_consolidado.json"

with open(archivo_salida, "w", encoding="utf-8") as f:
    json.dump(
        {
            "total_vacantes": len(vacantes_consolidadas),
            "vacantes": vacantes_consolidadas
        },
        f,
        ensure_ascii=False,
        indent=2
    )

print("Archivo generado:", archivo_salida)


Archivo generado: vacantes_colombia_consolidado.json


In [ ]:
files.download(archivo_salida)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ahora cargamos el archivo generado anteriormente y lo cargamos a una ubicacion para realizar el proceso de ajustes y dejar listo para crear los Embddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INPUT_PATH = "/content/drive/MyDrive/DPS (1)/Vacantes-21Dic2025/vacantes_colombia_consolidado.json"
OUTPUT_FILE = "/content/drive/MyDrive/DPS (1)/Vacantes-21Dic2025/vacantes_embeddings_ready.json"

In [ ]:
import json
import os
import unicodedata

def normalizar_texto(texto):
    if not texto:
        return ""
    texto = texto.strip().upper()
    texto = unicodedata.normalize('NFD', texto)
    texto = texto.encode('ascii', 'ignore').decode('utf-8')
    return texto

def construir_texto_embedding(v):
    departamento = normalizar_texto(v.get("DEPARTAMENTO", ""))
    municipio = normalizar_texto(v.get("MUNICIPIO", ""))
    descripcion = v.get("DESCRIPCION_VACANTE", "").strip()

    return (
        f"Este empleo está ubicado en el municipio de {municipio}, "
        f"departamento de {departamento}. "
        f"Descripción del cargo: {descripcion}"
    )

def extraer_prestadores(v):
    prestadores = []
    for p in v.get("DETALLES_PRESTADOR", []):
        prestadores.append({
            "nombre": p.get("NOMBRE_PRESTADOR", "").strip(),
            "url": p.get("URL_DETALLE_VACANTE", "").strip()
        })
    return prestadores

documentos = []

def procesar_archivo_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    vacantes = data.get("vacantes", [])

    for v in vacantes:
        documentos.append({
            "text": construir_texto_embedding(v),
            "metadata": {
                "departamento": normalizar_texto(v.get("DEPARTAMENTO")),
                "municipio": normalizar_texto(v.get("MUNICIPIO")),
                "rango_salarial": v.get("RANGO_SALARIAL", ""),
                "prestadores": extraer_prestadores(v)
            }
        })

# --- Lógica principal ---
if os.path.isdir(INPUT_PATH):
    print("📂 Procesando carpeta de JSON...")
    for filename in os.listdir(INPUT_PATH):
        if filename.endswith(".json"):
            procesar_archivo_json(os.path.join(INPUT_PATH, filename))
else:
    print("📄 Procesando archivo JSON único...")
    procesar_archivo_json(INPUT_PATH)

print(f"✅ Documentos generados: {len(documentos)}")


📄 Procesando archivo JSON único...
✅ Documentos generados: 143978


In [ ]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(documentos, f, ensure_ascii=False, indent=2)

print(f"Archivo generado en: {OUTPUT_FILE}")


Archivo generado en: /content/drive/MyDrive/DPS (1)/Vacantes-21Dic2025/vacantes_embeddings_ready.json
